# Initialize Hashicorp Vault cluster

- [docker container](https://hub.docker.com/_/vault)
- [Vault configuration file](https://developer.hashicorp.com/vault/docs/configuration)
- [envsubst manual](https://www.gnu.org/software/gettext/manual/html_node/envsubst-Invocation.html)
- [Docker Compose: Use a pre-existing network](https://docs.docker.com/compose/networking/#use-a-pre-existing-network)
- [xargs manual](https://man7.org/linux/man-pages/man1/xargs.1.html)
- [jq manual](https://devdocs.io/jq/)
- [tee manual](https://man7.org/linux/man-pages/man1/tee.1.html)
- [tr manual](https://man7.org/linux/man-pages/man1/tr.1.html)
- [sed manual](https://man7.org/linux/man-pages/man1/sed.1.html)

In [ ]:
echo $GNUPGHOME
echo $PASSWORD_STORE_DIR
echo $EXT_RUNTIME_DIR

In [ ]:
export NODE=vault1 && echo $NODE

In [ ]:
mkdir --parent --verbose $RUNTIME_DIR/$NODE/{config,logs,data}

In [ ]:
cat $WORK_DIR/vault/config.etmpl.hcl | envsubst | tee $RUNTIME_DIR/$NODE/config/config.hcl

In [ ]:
cat $WORK_DIR/vault/docker-compose.etmpl.yaml | envsubst | tee $RUNTIME_DIR/$NODE/docker-compose.yaml

In [ ]:
cd $RUNTIME_DIR/$NODE && sudo docker compose up -d $NODE

In [ ]:
export VAULT_ADDR=http://$NODE:8200 && echo $VAULT_ADDR

In [ ]:
ping -c 3 vault1

### NOTE
    This may take a few seconds

In [ ]:
vault operator init --format=json | tee $RUNTIME_DIR/init.json

In [ ]:
cat $RUNTIME_DIR/init.json | jq --raw-output '.unseal_keys_hex | to_entries[] | "echo \(.value) | pass add --multiline unseal-key-\(.key);"' |  xargs -t -d '\n' -I {} sh -c "{}"

In [ ]:
cat $RUNTIME_DIR/init.json | jq --raw-output '.root_token' | pass add --multiline root-token

In [ ]:
vault status

In [ ]:
vault operator unseal $(pass unseal-key-0)

In [ ]:
vault operator unseal $(pass unseal-key-1)

In [ ]:
vault operator unseal $(pass unseal-key-2)

In [ ]:
vault status